## Advanced Lane Finding Project Writeup


---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./output_images/undist.png "Undistorted"
[image2]: ./output_images/undist_test2.png "Road Transformed"
[image3]: ./output_images/binary.png "Binary Example"
[image4]: ./output_images/top_down.png "Warp Example"
[image5]: ./output_images/fit_line.png "Fit Visual"
[image6]: ./output_images/draw_lane.png "Output"
[video1]: ./output_project_video_oop.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Camera Calibration

The code for this step is contained in `1-Compute the camera calibration matri and distortion coefficients given a set of chessboard images` of the IPython notebook located in `Advanced Lane Detection with oop.ipynb`.

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  

![alt text][image1]

### Pipeline (single images)

#### 1. Example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
![alt text][image2]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

I used a combination of color and gradient thresholds to generate a binary image (thresholding steps at line #1 to line #62 in the cell `00-Define helpful functions & classes` in `Advanced Lane Detection with oop.ipynb`). I transformed RGB to HSV space to find yellow ((0,100,100) to (50,255,255)) and white ((20,0,180)to(255,80,255)) lines firstly, then transformed to HLS to find edges by applying sobelx on `l_channel`. Here's an example of my output for this step. 

![alt text][image3]

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `birdseye_warp()`, which appears in lines 31 through 49 in the cell `00-Define helpful functions & classes` in `Advanced Lane Detection with oop.ipynb` (output_images/examples/example.py) (or, for example, in the 3rd code cell of the IPython notebook).  The `birdseye_warp()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
vertices = [[img_size[0]//2-80,470],[200,img_size[1]],[img_size[0]-200,img_size[1]],[img_size[0]//2+80,470]]
src = np.float32(vertices)
offset = 250
dst = np.float32([[offset, 0], [offset,img_size[1]], 
                [img_size[0]-offset, img_size[1]], 
                [img_size[0]-offset, 0]])
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 560, 470      | 250, 0        | 
| 200, 720      | 250, 720      |
| 1080, 720     | 1030, 720      |
| 720, 470      | 1030, 0        |

The result is shown below:

![alt text][image4]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

To identify lane lines, I called the `pipeline()` and `birdseye_warp()` function to obtain a binary birds-eye view image with pixels. Then I developed code form line #130 to line #249 in the cell `00-Define helpful functions & classes` to find the pixels belong to lane-line by implementing sliding window method. Also, an algorithm searching around the parabola if lines were detected in the last frame for video cases was developed. Finally, I fit my lane lines with a 2nd order polynomial using `fitpoly()` like this:

![alt text][image5]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I defined fuction `measure_curvature()` in my code presented from line #251 to line #257 in  the cell `00-Define helpful functions & classes` and called it to calculate curvature of the lines.



#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines #260 to line #284 in my code in the cell `00-Define helpful functions & classes` in the function `draw_lane()`.  Here is an example of my result on a test image:

![alt text][image6]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./output_project_video_oop.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

In this project, I took the `cv2.calibrateCamera()` function to calibrate the camera which providing distortion coefficients and transform matrix by inputing many images with chessboard. After that, a test image of a real driving scenario was loaded and then undistorted by `cv2.undistort()`. The undistorted image was then transformed to binary birds eye view with color and gradients threshold and perspective trasfromation implemented. To fit a parabola of the lane-line, the lane-line pixels were detected by sliding window method firstly, and followed by `fitpoly()`. Finally, the curvature of each lines were calculated through `measure_curvature()`and the lane aera was drawn on the original image by `draw_lane()`. 

Finishing all steps for a single test image and proved reliable, those functions were packed in `process_image()` and `check()` was added to check if the lane-lines are reasonable according to the curvature, polynomial coefficients jump and line distance. The output video shows the smooth changing of lane aera between each frame. However, this program was not robust enough for `harder_challenge.mp4`. Lane-line distance became unreasonable small on shady bending mountain road. Some potential reason came in to my mind are: 
* The ROI is unchanged in my code while the lines on bending road are usually outside of that ROI.
* The mountain road is not falt which is contradictory to the assumption when developing the `birdseye_warp()` function.
* The color and gradients thresholds did not fit the scenarios.

The improvements could be:
* Develop dynamic ROI algorithm
* Implement radar to measure the gradient of the road. 
* Tune the color and gradients thresholds further.


